## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## 1. Camera Calibration

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import pickle
%matplotlib inline

def objectpoints(nx, ny):
    '''
    Prepare object points like (0,0,0) (1,0,0)....(8,5,0)
    '''
    
    objp = np.zeros((nx*ny,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)
    return objp

    
#Camera calibration function with the given number of corners (nx, ny) as input
def camera_calibration(directory, nx, ny):
    
 
   # Read in all the calibration images
    
    images = glob.glob(os.path.join(directory, 'calibration*.jpg'))
    
    # Arrays to store object points and image points from all the images.
    
    objpoints = [] # 3D points in the real world space
    imgpoints = [] # 2D points in the image plane
    
    for fname in images:
        img = cv2.imread(fname)
        
        # Convert image to grayscale 
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Find Chessboard corners using opencv 
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

  
        '''
        Some of the provided images to not have the full amount of the corners of the checssboard, 
        in this case we should look for a smaler amount of corner points (nx-1; ny-1 etc.)
        '''
        objp = objectpoints(nx, ny)
        if ret is False:
            ret, corners = cv2.findChessboardCorners(gray, (nx - 1, ny), None)
            objp = objectpoints(nx - 1, ny)
        if ret is False:
            ret, corners = cv2.findChessboardCorners(gray, (nx, ny - 1), None)
            objp = objectpoints(nx, ny - 1)
        if ret is False:
            ret, corners = cv2.findChessboardCorners(gray, (nx - 1, ny - 1), None)
            objp = objectpoints(nx - 1, ny - 1)
        if ret is False:
            ret, corners = cv2.findChessboardCorners(gray, (nx - 2, ny), None)
            objp = objectpoints(nx - 2, ny)
        if ret is False:
            ret, corners = cv2.findChessboardCorners(gray, (nx, ny - 2), None)
            objp = objectpoints(nx, ny - 2)
        if ret is False:
            ret, corners = cv2.findChessboardCorners(gray, (nx - 2, ny - 2), None)
            objp = objectpoints(nx - 2, ny - 2)
  
        # If found, add object points, image points
        if ret is True:
            
            # Append the points to the image points array
            imgpoints.append(corners)
            
            # Add the prepared object points array `objp` to the object points array
            objpoints.append(objp)
            
            # Plot the corner points
            img = cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
            plt.figure(figsize=(5,4))
            plt.imshow(img)
            plt.show()

        else:
            print('No corner point detected in: %s' %fname)
    
    # The object points and the detected corners (image points) will be used in order to calibrate the camera. 
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    
    return mtx, dist

mtx, dist = camera_calibration(directory = 'camera_cal/', nx = 9, ny = 6)



### Save the camera calibration results for later use

In [ ]:
cal_pickle = {}
cal_pickle['mtx'] = mtx
cal_pickle['dist'] = dist
pickle.dump( cal_pickle, open( "camera_cal/cal_pickle.p", "wb" ) )

### Load the camera calibration results

In [ ]:
# Read in the saved camera calibration results
dist_pickle = pickle.load( open( "camera_cal/cal_pickle.p", "rb" ) )
mtx = dist_pickle['mtx']
dist = dist_pickle['dist']

## 2. Correction of the Camera Distortion


In [ ]:
##  First the distortion co-efficient `dist`& camera matrix `mtx` will be applied on
##  the calibration images in order to test the proper undistortion
def cal_undistort(img, mtx, dist):
    # Apply the opencv function "undistort" in order to correct the distortion
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

# Load the calibration images and apply the "undostort" function on it
cal_imgs = glob.glob(os.path.join('camera_cal/', 'calibration*'))

# Function to plot the original and the undistored image
def plot_undist(cal_imgs):
    for fname in cal_imgs:
        img = cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB)
        undist = cal_undistort(img, mtx, dist)
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
        f.tight_layout()
        ax1.imshow(img)
        ax1.set_title('Original Image', fontsize=50)
        ax2.imshow(undist)
        ax2.set_title('Undistorted Image', fontsize=50)
        plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
        plt.show()

#Call the function
plot_undist(cal_imgs)



### Apply a distortion correction to raw images

In [ ]:
# Now we apply the undistort function on the "Test Images"
# Load the calibration images and apply the "undostort" function on it
test_imgs = glob.glob(os.path.join('test_images/', 'test*.jpg'))
plot_undist(test_imgs)

## 3. Create a thresholded binary image

In [ ]:
# Define a function that takes an image, gradient orientation,
# and threshold min / max values.
def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Return the result
    return binary_output

def thresholding(img, s_thresh=(170, 255), sx_thresh=(30, 255)):   
    # Apply Sobel in X direction 
    sobel_x = abs_sobel_thresh(img, 'x', 35, 255)
    
    # Select the white and yellow color from RGB colorspace
    rgb_white = cv2.inRange(img, (220, 220, 220), (255, 255, 255))
    rgb_yellow = cv2.inRange(img, (20, 100, 100), (50, 255, 255))
    
    # Select the white and yellow color from HSV colorspace
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    hsv_white = cv2.inRange(hsv, (220, 220, 220), (255, 255, 255))
    hsv_yellow = cv2.inRange(hsv, (20, 100, 100), (50, 255, 255))
    
    # Threshhold the S channel (Saturation) from HLS
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s = hls[:,:,2]
    s_output = np.zeros_like(s)
    s_output[(s > 120) & (s <= 255)] = 1
    
    # Select the white and yellow color from HLS colorspace
    hls_white = cv2.inRange(hls, (220,220,220), (255,255,255))
    hls_yellow = cv2.inRange(hls, (20, 100, 100), (50, 255, 255))

    # Combine the color and saturation masks 
    masked_rgb = cv2.bitwise_or(rgb_white , rgb_yellow) 
    masked_hsv = cv2.bitwise_or(hsv_white , hsv_yellow)
    masked_hls = cv2.bitwise_or(hls_white , hls_yellow)
    masked_color = cv2.bitwise_or(masked_rgb, masked_hsv, masked_hls,s_output)
    # Combined color mask (rgb, hsv, hls, saturation) and gradient (sobel) mask
    combined_mask = cv2.bitwise_or(masked_color, sobel_x)
    
    return combined_mask

In [ ]:
# Apply the thresholding on the test images
test_imgs = glob.glob(os.path.join('test_images/', 'test*.jpg'))

for fname in test_imgs:
    img = cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB)
    undist = cal_undistort(img, mtx, dist)
    masked = thresholding(undist)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(masked, cmap = 'gray')
    ax2.set_title('Undistorted Image', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    plt.show()

## 4. Perspective transform ("birds-eye view").
Undistorted images will be transformed in the "birds-eye view)

In [ ]:


# Load the test image with straight lines read the coordinates of the lines (create trapezium) as source points
test_img = './test_images/straight_lines1.jpg'
img = cv2.cvtColor(cv2.imread(test_img), cv2.COLOR_BGR2RGB)
undist_img = cal_undistort(img, mtx, dist)
plt.imshow (undist_img)

src = np.float32([(195, 720), (1100, 720), (715, 465), (565, 465)])
plt.plot(src[0][0], src[0][1],'.') # buttom left
plt.plot(src[1][0], src[1][1],'.') # buttom right
plt.plot(src[2][0], src[2][1],'.') # top right
plt.plot(src[3][0], src[3][1],'.') # top left

# Define the destination points (create rectangle) => Check if the destination points are OK after the Perspective transform
dst = np.float32([(295, 720), (990, 720), (990, 0), (295, 0)])

# Compute the perspective transform, M and the inverse M with given source and destination points:
def PerspectiveTransform(src, dst):
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    return M, Minv

# Get the transform matrices 
M, Minv = PerspectiveTransform(src, dst)

test_imgs = glob.glob(os.path.join('test_images/', 'straight_lines*.jpg'))
for fname in test_imgs:
    # Get the original image
    img = cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB)
    undist = cal_undistort(img, mtx, dist)    
    birdseye = cv2.warpPerspective(undist, M, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR)
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
    f.tight_layout()
    ax1.imshow(img)
    ax1.set_title('Original Image', fontsize=50)
    ax2.imshow(birdseye, cmap = 'gray')
    ax2.set_title('Birds-eye View', fontsize=50)
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
    plt.show()

## 4. Detect lane pixels and fit to find the lane boundary.

In [ ]:
def get_base_x (img, plot = False):
    # Take a histogram of the bottom half of the image
    # Compute histogram
    binary_warped = img.copy()
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    
    if plot == True:
        #Plot the histogram
        plt.plot(histogram)
        plt.autoscale(enable=True, axis='x', tight=True)
    
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    return leftx_base, rightx_base

test_img = './test_images/test5.jpg'
img_original = mpimg.imread(test_img) 
undist = cal_undistort(img_original, mtx, dist)    
img_warped = cv2.warpPerspective(undist, M, (img.shape[1], img.shape[0]), flags=cv2.INTER_LINEAR)
img_mask = thresholding (img_warped)


plt.figure();
plt.imshow(img_mask, cmap='gray');
plt.title('Masked, warped image');
plt.figure();

leftx_base, rightx_base = get_base_x(img_mask, plot = True)

## Implement Sliding Windows and Fit a Polynomial



In [ ]:
def sliding_window (img, leftx_base, rightx_base, plot = False):
    binary_warped = img.copy()
    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero (i.e. activated) pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base
        
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass
    
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
        
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    
    # Plot the sliding windows
    if plot == True:
        plt.figure(figsize=(10,10))
        fig = plt.figure()

        plt.imshow(out_img)
        plt.plot(left_fitx, ploty, color='yellow')
        plt.plot(right_fitx, ploty, color='yellow')
        plt.xlim(0, 1280)
        plt.ylim(720, 0)
            
    return left_fit, right_fit, out_img

left_fit, right_fit, out_img = sliding_window(img_mask, leftx_base, rightx_base, plot = True)

In [ ]:
def fit_continuous(left_fit, right_fit, img, plot = True):

    binary_warped = img.copy()
    # It's now much easier to find line pixels!
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 50
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) 
                      & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) 
                       & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each
    if len(leftx) == 0:
        left_fit_new =[]
    else:
        left_fit_new = np.polyfit(lefty, leftx, 2)
    
    
    if len(rightx) == 0:
        right_fit_new =[]
    else:
        right_fit_new = np.polyfit(righty, rightx, 2)
        
    if plot == True:
    
        # Generate x and y values for plotting
        ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

        # Create an image to draw on and an image to show the selection window
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
        window_img = np.zeros_like(out_img)
        # Color in left and right line pixels
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

        # Generate a polygon to illustrate the search window area
        # And recast the x and y points into usable format for cv2.fillPoly()
        left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
        left_line_pts = np.hstack((left_line_window1, left_line_window2))
        right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
        cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
        result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
   
        plt.imshow(result)
        plt.plot(left_fitx, ploty, color='yellow')
        plt.plot(right_fitx, ploty, color='yellow')
        plt.xlim(0, 1280)
        plt.ylim(720, 0)
    
    return left_fit_new, right_fit_new

fit_continuous(left_fit, right_fit, img_mask, plot = True)